### **Import lib**

In [1]:
import pandas as pd
import numpy as np
from dateutil import tz
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

### **1. Reading data**

**Hong Nhung crawled data**

In [2]:
hn1 = pd.read_csv(r"D:\UIT\DS200.P21_Big_Data-4\Dataset\Raw Data\User Videos\HN\momandbaby_food_videos_1.csv")
hn2 = pd.read_csv(r"D:\UIT\DS200.P21_Big_Data-4\Dataset\Raw Data\User Videos\HN\momandbaby_food_videos_1.csv")
hn3 = pd.read_csv(r"D:\UIT\DS200.P21_Big_Data-4\Dataset\Raw Data\User Videos\HN\momandbaby_food_videos_1.csv")
hn4 = pd.read_csv(r"D:\UIT\DS200.P21_Big_Data-4\Dataset\Raw Data\User Videos\HN\momandbaby_food_videos_1.csv")
hn = pd.concat([hn1, hn2, hn3, hn4])

hn_user_name = pd.read_csv(r"D:\UIT\DS200.P21_Big_Data-4\Dataset\Raw Data\User name\momandbaby_food.csv")

**Thien Phuc crawled data**

In [3]:
tp1 = pd.read_csv(r"D:\UIT\DS200.P21_Big_Data-4\Dataset\Raw Data\User Videos\TPL\health_techandhousehold_videos_1.csv")
tp2 = pd.read_csv(r"D:\UIT\DS200.P21_Big_Data-4\Dataset\Raw Data\User Videos\TPL\health_techandhousehold_videos_1.csv")
tp3 = pd.read_csv(r"D:\UIT\DS200.P21_Big_Data-4\Dataset\Raw Data\User Videos\TPL\health_techandhousehold_videos_1.csv")
tp4 = pd.read_csv(r"D:\UIT\DS200.P21_Big_Data-4\Dataset\Raw Data\User Videos\TPL\health_techandhousehold_videos_1.csv")
tp = pd.concat([tp1, tp2, tp3, tp4])

tp_user_name = pd.read_csv(r"D:\UIT\DS200.P21_Big_Data-4\Dataset\Raw Data\User name\health_techandhousehold.csv")

**Ngoc Mai crawled data**

In [ ]:
nm1 = pd.read_csv(r"D:\UIT\DS200.P21_Big_Data-4\Dataset\Raw Data\User Videos\NM\fashion_beauty_videos_1.csv")
nm2 = pd.read_csv(r"D:\UIT\DS200.P21_Big_Data-4\Dataset\Raw Data\User Videos\NM\fashion_beauty_videos_2.csv")
nm3 = pd.read_csv(r"D:\UIT\DS200.P21_Big_Data-4\Dataset\Raw Data\User Videos\NM\fashion_beauty_videos_3.csv")
nm4 = pd.read_csv(r"D:\UIT\DS200.P21_Big_Data-4\Dataset\Raw Data\User Videos\NM\fashion_beauty_videos_4.csv")
nm = pd.concat([nm1, nm2, nm3, nm4])

nm_user_name = pd.read_csv(r"D:\UIT\DS200.P21_Big_Data-4\Dataset\Raw Data\User name\beauty_fashion.csv")

**Merging data**

In [5]:
total_user_name = pd.concat([hn_user_name, tp_user_name, nm_user_name])
total_user_name.reset_index(inplace=True)
total_user_name.drop(columns= ['index'], inplace = True)
total_user_name = total_user_name.rename(columns= {'username':'user_name'})

In [6]:
df = pd.concat([hn, tp, nm])
df.reset_index(inplace = True)

### **2. Lọc trùng các user_name**

**Drop Duplicates**

In [7]:
total_user_name.drop_duplicates(inplace = True)

**Grouping topic**

In [8]:
total_user_name = total_user_name.groupby('user_name')['topic'].agg(lambda x: list(set(x))).reset_index()

In [9]:
total_user_name.shape[0]

392

### **3. Lọc các video không đủ yêu cầu về snapshot**

#### *Chuyển dữ liệu các cột thời gian thành dữ liệu datetime*

In [10]:
def convert_to_vietnam_time(utc_time):
    from_zone = tz.tzutc()
    to_zone = tz.gettz('Asia/Ho_Chi_Minh')
    utc = pd.to_datetime(utc_time).replace(tzinfo=from_zone)
    return utc.astimezone(to_zone)

In [11]:
df['vid_postTime'] = df['vid_postTime'].apply(convert_to_vietnam_time)
df['vid_scrapeTime'] = df['vid_scrapeTime'].apply(convert_to_vietnam_time)

#### *Sắp xếp các  dòng dữ liệu theo từng vid_id*

In [12]:
df.sort_values(['user_name', 'vid_id', 'vid_scrapeTime'], ascending=True, inplace=True)

#### *Lọc dòng dữ liệu trùng*

In [13]:
df.shape[0]

38148

In [14]:
df.drop_duplicates(inplace = True)
df.reset_index(inplace = True)
df.shape[0]

25554

#### *Số lượng sample của mỗi lượng snapshot*

In [15]:
# Đếm số snapshot của mỗi video
video_counts = df.groupby('vid_id').size().reset_index(name='snapshot_counts')

# Merge lại để kiểm tra rõ ràng
df_with_count = df.merge(video_counts, on='vid_id')

# Kiểm tra phân phối
df_with_count['snapshot_counts'].value_counts().sort_index()

snapshot_counts
1     4645
2     3178
3     2913
4    14620
5       20
6       12
7       14
8      152
Name: count, dtype: int64

#### *Xóa các video chỉ có 1-2 snapshot*

In [16]:
# Đếm số mốc thời gian của mỗi video
video_counts = df.groupby(['user_name', 'vid_id'])['vid_scrapeTime'].count().reset_index()
video_counts = video_counts.rename(columns={'vid_scrapeTime': 'snapshot_count'})

# Lọc các video chỉ có 1-2 snapshot
invalid_videos = video_counts[video_counts['snapshot_count'] <= 2]

# Các video có đủ 4 snapshot 
valid_videos = video_counts[video_counts['snapshot_count'] == 4]
valid_data = df.merge(valid_videos[['user_name', 'vid_id']], on=['user_name', 'vid_id'], how='inner')

# Các video chỉ có 3 snapshot 
snapshot_3 = video_counts[video_counts['snapshot_count'] == 3]
snapshot_3 = df.merge(snapshot_3[['user_name', 'vid_id']], on=['user_name', 'vid_id'], how='inner')

# Các video có từ 5 snapshot trở lên 
morethan_5_snapshot = video_counts[video_counts['snapshot_count'] >= 5]
morethan_5_snapshot = df.merge(morethan_5_snapshot[['user_name', 'vid_id']], on=['user_name', 'vid_id'], how='inner')

# Reset index và drop các cột dư thừa 
valid_data.drop(columns = ['level_0', 'index'], inplace = True)
snapshot_3.drop(columns = ['level_0', 'index'], inplace = True)


# In ra số lượng sample của mối loại dữ liệu 
print("Tổng số lượng sample: ", df.shape[0])
print("Số sample của video có ít hơn 2 snapshot: ", invalid_videos.shape[0])
print("Số sample của video có đủ 4 snapshot: ", valid_data.shape[0])
print("Số sample của video có 3 snapshot: ", snapshot_3.shape[0])
print("Số sample của video có nhiều hơn 5 snapshot: ", morethan_5_snapshot.shape[0])

Tổng số lượng sample:  25554
Số sample của video có ít hơn 2 snapshot:  6234
Số sample của video có đủ 4 snapshot:  14620
Số sample của video có 3 snapshot:  2913
Số sample của video có nhiều hơn 5 snapshot:  198


### **4. Lọc các snapshot có khoảng cách thời gian quá gần**

#### *Hàm trích xuất khoảng thời gian giữa các video*

In [17]:
from datetime import timedelta

def select_latest_4_snapshots(df):
    filtered_dfs = []

    # Đảm bảo datetime đúng
    df = df.sort_values(['vid_id', 'vid_scrapeTime'], ascending=[True, False])

    for vid_id, group in df.groupby('vid_id'):
        selected = []
        used_times = []

        for _, row in group.iterrows():
            current_time = row['vid_scrapeTime']

            # Nếu chưa chọn snapshot nào, chọn snapshot đầu tiên
            if not used_times:
                selected.append(row)
                used_times.append(current_time)
            else:
                # Kiểm tra thời gian tối thiểu 1 ngày với các snapshot đã chọn
                if all(abs((current_time - prev_time).days) >= 1 for prev_time in used_times):
                    selected.append(row)
                    used_times.append(current_time)

            # Nếu đã đủ 4 snapshot, dừng
            if len(selected) == 4:
                break

        if selected:
            filtered_dfs.append(pd.DataFrame(selected))

    return pd.concat(filtered_dfs, ignore_index=True)


In [18]:
filtered_morethan_5_snapshot = select_latest_4_snapshots(morethan_5_snapshot)
filtered_morethan_5_snapshot.sort_values(['vid_id', 'vid_scrapeTime'], ascending=True, inplace=True)
filtered_morethan_5_snapshot.drop(columns = ['level_0', 'index'], inplace = True)
filtered_morethan_5_snapshot.shape[0]

108

#### *Gộp các sample tiêu chuẩn - đủ 4 snapshot*

In [19]:
valid_data  = pd.concat([valid_data, filtered_morethan_5_snapshot])
valid_data.sort_values(['vid_id', 'vid_scrapeTime'], ascending=True, inplace=True)
valid_data.reset_index(inplace=True)
valid_data.drop(columns=['index'], inplace=True)
valid_data.shape[0]

14728

### **5. Lọc các user_name unique**

In [20]:
unique_user_name = total_user_name[total_user_name['user_name'].isin(valid_data['user_name'].unique())]
unique_user_name.shape[0]

191

**Merge topic vào valid_data**

In [21]:
valid_data = valid_data.merge(unique_user_name, on=['user_name'], how='left')

### **6. Lưu các file data**

In [23]:
valid_data.to_csv(r"D:\UIT\DS200.P21_Big_Data-4\Dataset\Raw Data\Merged Data\training_data.csv")
snapshot_3.to_csv(r"D:\UIT\DS200.P21_Big_Data-4\Dataset\Raw Data\Merged Data\streaming_data.csv")
unique_user_name.to_csv(r"D:\UIT\DS200.P21_Big_Data-4\Dataset\Raw Data\Merged Data\total_user_name.csv")